In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords 
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df = pd.read_csv('yelp_review.csv')

In [3]:
df.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


In [4]:
#reading dataset with information about business(type)
data_business=pd.read_csv('yelp_business.csv')

In [5]:
data_business.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,FYWN1wneV18bWNgQjJ2GNg,"""Dental by Design""",NaN,"""4855 E Warner Rd, Ste B9""",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,22,1,Dentists;General Dentistry;Health & Medical;Or...
1,He-G7vWjzVUysIKrfNbPUQ,"""Stephen Szabo Salon""",NaN,"""3101 Washington Rd""",McMurray,PA,15317,40.291685,-80.104900,3.0,11,1,Hair Stylists;Hair Salons;Men's Hair Salons;Bl...
2,KQPW8lFf1y5BT2MxiSZ3QA,"""Western Motor Vehicle""",NaN,"""6025 N 27th Ave, Ste 1""",Phoenix,AZ,85017,33.524903,-112.115310,1.5,18,1,Departments of Motor Vehicles;Public Services ...
3,8DShNS-LuFqpEWIp0HxijA,"""Sports Authority""",NaN,"""5000 Arizona Mills Cr, Ste 435""",Tempe,AZ,85282,33.383147,-111.964725,3.0,9,0,Sporting Goods;Shopping
4,PfOCPjBrlQAnz__NXj9h_w,"""Brick House Tavern + Tap""",NaN,"""581 Howe Ave""",Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,116,1,American (New);Nightlife;Bars;Sandwiches;Ameri...


In [6]:
business=data_business[data_business['categories'].str.match('Restaurant')].copy()

In [7]:
# merging reviews and businnes based on business_id
f1=pd.merge(df, business, on='business_id')

In [8]:
f1.head()

,review_id,user_id,business_id,stars_x,date,text,useful,funny,cool,name,...,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories
0,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
1,VrhTbFhgUa4zmx6jfqp_hQ,gKYaocnFD8Zi6rMt_mT-cw,VR6GpWIda3SfvPC-lg9H3w,5,2015-01-31,Highly recommend checking this place out! Tuc...,1,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
2,XMJ_sHKVrtqJZ4IXA0I9wA,Vp_C7oIxeztbd2PchNBOHw,VR6GpWIda3SfvPC-lg9H3w,4,2011-10-23,This restaurant is part of the wave of trendy ...,1,0,4,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
3,MhA0-xaUx4o4Pl_GQUr_ZQ,tB6vkXCS_8PIh-T9g8t6IQ,VR6GpWIda3SfvPC-lg9H3w,5,2011-07-22,"It's in a sketchy area, but it's upscale and l...",2,0,1,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
4,ziRcRyoy96J1Wj_WN-pM2Q,i1XAHltBpZC5zP5skcwVGw,VR6GpWIda3SfvPC-lg9H3w,4,2014-11-01,A local friend introduced me to this jewel res...,3,2,2,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian


Important features user_id, text, stars_x (rating from user)

In [9]:
df_in = f1.iloc[0:200, :]

In [10]:
df_in.head()

,review_id,user_id,business_id,stars_x,date,text,useful,funny,cool,name,...,address,city,state,postal_code,latitude,longitude,stars_y,review_count,is_open,categories
0,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
1,VrhTbFhgUa4zmx6jfqp_hQ,gKYaocnFD8Zi6rMt_mT-cw,VR6GpWIda3SfvPC-lg9H3w,5,2015-01-31,Highly recommend checking this place out! Tuc...,1,0,0,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
2,XMJ_sHKVrtqJZ4IXA0I9wA,Vp_C7oIxeztbd2PchNBOHw,VR6GpWIda3SfvPC-lg9H3w,4,2011-10-23,This restaurant is part of the wave of trendy ...,1,0,4,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
3,MhA0-xaUx4o4Pl_GQUr_ZQ,tB6vkXCS_8PIh-T9g8t6IQ,VR6GpWIda3SfvPC-lg9H3w,5,2011-07-22,"It's in a sketchy area, but it's upscale and l...",2,0,1,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian
4,ziRcRyoy96J1Wj_WN-pM2Q,i1XAHltBpZC5zP5skcwVGw,VR6GpWIda3SfvPC-lg9H3w,4,2014-11-01,A local friend introduced me to this jewel res...,3,2,2,"""Tuck Shop""",...,"""4662 Rue Notre-Dame O""",Montréal,QC,H4C 1S7,45.472902,-73.588321,4.5,50,1,Restaurants;Canadian (New);Italian


In [11]:
df_in.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 21 columns):
review_id       200 non-null object
user_id         200 non-null object
business_id     200 non-null object
stars_x         200 non-null int64
date            200 non-null object
text            200 non-null object
useful          200 non-null int64
funny           200 non-null int64
cool            200 non-null int64
name            200 non-null object
neighborhood    200 non-null object
address         200 non-null object
city            200 non-null object
state           200 non-null object
postal_code     200 non-null object
latitude        200 non-null float64
longitude       200 non-null float64
stars_y         200 non-null float64
review_count    200 non-null int64
is_open         200 non-null int64
categories      200 non-null object
dtypes: float64(3), int64(6), object(12)
memory usage: 34.4+ KB


# Extracting only reviews

In [12]:
#don't you think we can need user_id to suggests anothers users ?
review_df = df_in[['stars_x', 'text']]

In [13]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 2 columns):
stars_x    200 non-null int64
text       200 non-null object
dtypes: int64(1), object(1)
memory usage: 4.7+ KB


# Data Cleaning

In [14]:
review_arr =  review_df.as_matrix()

C:\Users\John\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [15]:
review_arr.shape

(200, 2)

In [16]:
clean_ar = np.empty(shape=(review_arr.shape), dtype=object)

In [17]:
type(clean_ar[0][1])

NoneType

In [18]:
def data_clean(arr, clean_ar):
    for i in range(len(arr)):
        sub_text =  re.sub("[^a-zA-Z]", " ", arr[i][1])
        l_case = sub_text.lower()
        words = l_case.split()
        stop = set(stopwords.words("english"))
        m_words = [wr for wr in words if not wr in stop]
        text = (" ".join(m_words))
        clean_ar[i][1] = text
        clean_ar[i][0] = arr[i][0]
        
    
    return clean_ar

In [19]:
data = data_clean(review_arr, clean_ar)
clean_df = pd.DataFrame(data)

In [20]:
clean_df.head()

,0,1
0,5,small unassuming place changes menu every ofte...
1,5,highly recommend checking place tuck shop reco...
2,4,restaurant part wave trendy chef centric bistr...
3,5,sketchy area upscale lovely small bistro type ...
4,4,local friend introduced jewel restaurant son m...


In [21]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
0    200 non-null object
1    200 non-null object
dtypes: object(2)
memory usage: 3.2+ KB


# Vectorization

In [22]:
vectorizer = TfidfVectorizer(norm = 'l2',min_df = 0, use_idf = True, smooth_idf = False, sublinear_tf = True, \
                             ngram_range=(1,2))


train_vect = vectorizer.fit_transform(clean_df.iloc[:,1])
train_vect = train_vect.toarray()
print (train_vect.shape)

(200, 15137)


In [23]:
comp_vect = vectorizer.transform(clean_df.iloc[:,1])
comp_vect = comp_vect.toarray()
print (comp_vect.shape)

(200, 15137)


In [24]:
#get cosine similarity
csr_sim = cosine_similarity(comp_vect,train_vect)